In [1]:
import numpy as np
from collections import defaultdict

class Agent:

    def __init__(self, nA=6):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.epsilon = 0.0005  # Epsilon value, probability of taking on a random action in a epsilon-greedy policy (epsion = 0 means no randomness)
        self.alpha = 0.07      # Alpha value, the Step Size
        self.gamma = 0.999     # Gamma value, the Discount Rate, (gamma = 0 means only results from most immediate reward is being used)
                

    def select_action(self, state):
        """ Given the state, select an action.
        """
        policy_state = ((np.ones(self.nA) * self.epsilon) / self.nA)
        greedy_Q = np.argmax(self.Q[state])
        policy_state[greedy_Q] = 1 - self.epsilon + (self.epsilon / self.nA)
        action = np.random.choice(self.nA, p=policy_state)
        return action  

    def step(self, state, action, reward, next_state, done):
        """ Update the agent's knowledge, using the most recently sampled tuple.
        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        next_action = self.select_action(next_state)
        Goft = reward + ((self.gamma) * self.Q[next_state][next_action])
        self.Q[state][action] += self.alpha * (Goft - self.Q[state][action])  

In [2]:
from collections import deque
import sys
import math
import numpy as np

def interact(env, agent, num_episodes=20000, window=100):
    """ Monitor agent's performance.
    
    Params
    ======
    - env: instance of OpenAI Gym's Taxi-v1 environment
    - agent: instance of class Agent (see Agent.py for details)
    - num_episodes: number of episodes of agent-environment interaction
    - window: number of episodes to consider when calculating average rewards

    Returns
    =======
    - avg_rewards: deque containing average rewards
    - best_avg_reward: largest value in the avg_rewards deque
    """
    # initialize average rewards
    avg_rewards = deque(maxlen=num_episodes)
    # initialize best average reward
    best_avg_reward = -math.inf
    # initialize monitor for most recent rewards
    samp_rewards = deque(maxlen=window)
    # for each episode
    for i_episode in range(1, num_episodes+1):
        # begin the episode
        state = env.reset()
        # initialize the sampled reward
        samp_reward = 0
        while True:
            # agent selects an action
            action = agent.select_action(state)
            # agent performs the selected action
            next_state, reward, done, _ = env.step(action)
            # agent performs internal updates based on sampled experience
            agent.step(state, action, reward, next_state, done)
            # update the sampled reward
            samp_reward += reward
            # update the state (s <- s') to next time step
            state = next_state
            if done:
                # save final sampled reward
                samp_rewards.append(samp_reward)
                break
        if (i_episode >= 100):
            # get average reward from last 100 episodes
            avg_reward = np.mean(samp_rewards)
            # append to deque
            avg_rewards.append(avg_reward)
            # update best average reward
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
        # monitor progress
        print("\rEpisode {}/{} || Best average reward {}".format(i_episode, num_episodes, best_avg_reward), end="")
        sys.stdout.flush()
        # check if task is solved (according to OpenAI Gym)
        if best_avg_reward >= 9.7:
            print('\nEnvironment solved in {} episodes.'.format(i_episode), end="")
            break
        if i_episode == num_episodes: print('\n')
    return avg_rewards, best_avg_reward

In [3]:
import gym
import numpy as np

env = gym.make('Taxi-v2')
agent = Agent()
for i in range(50):
    avg_rewards, best_avg_reward = interact(env, agent)

Episode 20000/20000 || Best average reward 9.224

Episode 20000/20000 || Best average reward 9.45

Episode 20000/20000 || Best average reward 9.32

Episode 20000/20000 || Best average reward 9.31

Episode 20000/20000 || Best average reward 9.33

Episode 20000/20000 || Best average reward 9.35

Episode 20000/20000 || Best average reward 9.4

Episode 20000/20000 || Best average reward 9.29

Episode 20000/20000 || Best average reward 9.17

Episode 20000/20000 || Best average reward 9.33

Episode 20000/20000 || Best average reward 9.32

Episode 20000/20000 || Best average reward 9.26

Episode 20000/20000 || Best average reward 9.22

Episode 20000/20000 || Best average reward 9.3

Episode 20000/20000 || Best average reward 9.26

Episode 20000/20000 || Best average reward 9.35

Episode 20000/20000 || Best average reward 9.23

Episode 20000/20000 || Best average reward 9.32

Episode 20000/20000 || Best average reward 9.36

Episode 20000/20000 || Best average reward 9.27

Episode 20000/20000 |